In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


# **1.Lấy dữ liệu comments**

In [ ]:
import requests
import json
import csv


In [ ]:

def get_all_comments(post_id, access_token):
    comments_data = []
    next_url = f"https://graph.facebook.com/{post_id}/comments?access_token={access_token}"

    while next_url:
        response = requests.get(next_url)
        data = response.json()
        if "data" in data:
            comments_data.extend(data["data"])
        if "paging" in data and "next" in data["paging"]:
            next_url = data["paging"]["next"]
        else:
            next_url = None

    return comments_data

# Thay thế POST_ID và ACCESS_TOKEN bằng giá trị thích hợp của bạn
POST_ID = 'pfbid02RTLY81bKvyjp3sDsu1owZ98vkmZbNkMa5eSo6VjUxrw4QZY1TCx5cmYsWYMac6VNl'
ACCESS_TOKEN = 'EAABwzLixnjYBO4k1s2TYKguBgwapSsTo4uG4NNVIO1gKlnXZBS36VABgcBJk3rBZBUUMt1IDv2QHP5YNSO9KrDpqoGXwLB0w31padMUT6JlWMJMISjrl7ND2ZCMm7SxOiEDoCr7UZCndXS6S8cvBePOxGduSGkR49oWh42ZAstlg7k9cL4chTu31JUWMokxCDPZCaAAfaa8IAZD'

comments_data = get_all_comments(POST_ID, ACCESS_TOKEN)

if comments_data:
    # Ghi vào tệp CSV
    csv_path = '/content/drive/MyDrive/project_facebook_bigdata/all_data_file/comments.csv'
    with open(csv_path, 'w', encoding='utf-8', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(['message', 'created_time', 'from_name', 'from_id'])  # Tiêu đề cột
        for comment in comments_data:
            csv_writer.writerow([comment.get('message', ''), comment.get('created_time', ''), comment.get('from', {}).get('name', ''), comment.get('from', {}).get('id', '')])

    # Ghi vào tệp JSON
    json_path = '/content/drive/MyDrive/project_facebook_bigdata/all_data_file/comments.json'
    with open(json_path, 'w', encoding='utf-8') as json_file:
        json.dump(comments_data, json_file, ensure_ascii=False, indent=4)

    print("Dữ liệu bình luận đã được ghi vào comments.csv và comments.json trên Google Drive")
    print("Đường dẫn tới tệp CSV:", csv_path)
    print("Đường dẫn tới tệp JSON:", json_path)
else:
    print("Không tìm thấy thông tin về bình luận cho bài viết này.")


Dữ liệu bình luận đã được ghi vào comments.csv và comments.json trên Google Drive
Đường dẫn tới tệp CSV: /content/drive/MyDrive/project_facebook_bigdata/all_data_file/comments.csv
Đường dẫn tới tệp JSON: /content/drive/MyDrive/project_facebook_bigdata/all_data_file/comments.json


# **2.Lấy dữ liệu reactions**

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%pip install facebook_scraper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.8/293.8 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.5/131.5 kB 15.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.4/83.4 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 10.5 MB/s eta 0:00:00
  Created wheel for demjson3: filename=demjson3-3.0.6-py3-none-any.whl size=75281 sha256=d4c27dfe8aa7e171900035ca34be8df36968260871166c9f0481878e40583039
  Stored in directory: /root/.cache/pip/wheels/2d/32/af/75c810e14c7ca0df40c20c662fcc7f33d1e055937b4e452cc1
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1256 sha256=b15b4dac323cf5b4e1185068d6a0fe122032e2d25948920355307bed6dda3279
  Stored in directory:

In [ ]:
import facebook_scraper as fs
import pandas as pd

In [ ]:
POST_ID = "pfbid02RTLY81bKvyjp3sDsu1owZ98vkmZbNkMa5eSo6VjUxrw4QZY1TCx5cmYsWYMac6VNl"
COOKIE_PATH= "/content/drive/MyDrive/test scap/facebook.com_cookies.txt"

MAX_COMMENTS = True

# get the post (this gives a generator)
gen = fs.get_posts(
    post_urls=[POST_ID],
    options={"reactions": True, "allow_extra_requests": True}, # option có những gì thì đọc docs, nhiều quá không viết hết được https://github.com/kevinzg/facebook-scraper
   cookies=COOKIE_PATH) # ở đay chỉ lấy data về reactions thôi nhé


# take 1st element of the generator which is the post we requested
post = next(gen)


In [ ]:
len(post)

52

In [ ]:
post

{'original_request_url': 'pfbid02RTLY81bKvyjp3sDsu1owZ98vkmZbNkMa5eSo6VjUxrw4QZY1TCx5cmYsWYMac6VNl',
 'post_url': 'https://facebook.com/story.php?story_fbid=pfbid02Rm94PWtnw6jonDH5rTdwLvp6zKdLYJUhhfndipFboc1dwNnQCM8fjU6harF14t2nl&id=100064868580919',
 'post_id': 'pfbid02Rm94PWtnw6jonDH5rTdwLvp6zKdLYJUhhfndipFboc1dwNnQCM8fjU6harF14t2nl',
 'text': '🧋CUNG HOÀNG ĐẠO CỦA CÁC MÓN PHÚC LONG\nCác cung hoàng đạo vào xác nhận đi nào!!!\n\n#phuclong',
 'post_text': '🧋CUNG HOÀNG ĐẠO CỦA CÁC MÓN PHÚC LONG\nCác cung hoàng đạo vào xác nhận đi nào!!!\n\n#phuclong',
 'shared_text': '',
 'original_text': None,
 'time': datetime.datetime(2023, 10, 17, 11, 35),
 'timestamp': 1697542500,
 'image': 'https://scontent-iad3-1.xx.fbcdn.net/v/t39.30808-6/393350272_716392107199736_1443214181380341480_n.jpg?stp=cp0_dst-jpg_e15_fr_q65&_nc_cat=110&ccb=1-7&_nc_sid=5f2048&efg=eyJpIjoidCJ9&_nc_ohc=Ou-yYf4NgQAAX9vd5fR&_nc_oc=AQkevGSuVSg0eRTY6VAbDrWv6YwygCrTv9PZ7SeL--gqyNeFf5Qc7QoGfs9BNyob8kc&_nc_ht=scontent-iad3-1.xx&oh

In [ ]:
import json

# Tạo một từ điển chứa các trường bạn muốn lưu
data_to_save = {
    "reactions": post['reactions'],
    "reaction_count": post['reaction_count'],
    "comments": post['comments'],
    "shares": post['shares']
}

# Mở tệp JSON để lưu dữ liệu
with open("/content/drive/MyDrive/project_facebook_bigdata/all_data_file/reactions_data.json", "w") as json_file:
    json.dump(data_to_save, json_file)

# Đã lưu dữ liệu vào tệp "post_data.json"


In [ ]:
# post_df = pd.DataFrame([post])

In [ ]:
# post_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 52 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   original_request_url           1 non-null      object        
 1   post_url                       1 non-null      object        
 2   post_id                        1 non-null      object        
 3   text                           1 non-null      object        
 4   post_text                      1 non-null      object        
 5   shared_text                    1 non-null      object        
 6   original_text                  0 non-null      object        
 7   time                           1 non-null      datetime64[ns]
 8   timestamp                      1 non-null      int64         
 9   image                          1 non-null      object        
 10  image_lowquality               1 non-null      object        
 11  images                 

In [ ]:
# post_df.head()

,original_request_url,post_url,post_id,text,post_text,shared_text,original_text,time,timestamp,image,...,w3_fb_url,reactions,reaction_count,with,page_id,sharers,image_id,image_ids,was_live,fetched_time
0,pfbid02RTLY81bKvyjp3sDsu1owZ98vkmZbNkMa5eSo6Vj...,https://facebook.com/story.php?story_fbid=pfbi...,pfbid02Rm94PWtnw6jonDH5rTdwLvp6zKdLYJUhhfndipF...,🧋CUNG HOÀNG ĐẠO CỦA CÁC MÓN PHÚC LONG\nCác cun...,🧋CUNG HOÀNG ĐẠO CỦA CÁC MÓN PHÚC LONG\nCác cun...,,None,2023-10-17 11:35:00,1697542500,https://scontent-iad3-1.xx.fbcdn.net/v/t39.308...,...,https://www.facebook.com/story.php?story_fbid=...,"{'thích': 2815, 'yêu thích': 186, 'haha': 649,...",3878,None,430176977051947,None,716392110533069,[716392110533069],False,2023-10-31 13:08:41.925273


In [ ]:
# new_path = "/content/drive/MyDrive/project_facebook_bigdata/all_data_file/reaction_data.xlsx"
# # Ghi tệp Excel vào đường dẫn mới
# post_df.to_excel(new_path, index=False)
# # In ra đường dẫn mới
# print(new_path)


/content/drive/MyDrive/project_facebook_bigdata/all_data_file/reaction_data.xlsx
